---
title: Define Promotion Usage Metrics
---
# 3. Define Promotion Usage Metrics

We measure promotion intensity using two complementary metrics:

### 3.1 Discount Usage
From `transactions`, we calculate what % of items were purchased with **any** discount:
- `retail_disc` > 0 (sale price)
- `coupon_disc` > 0 (manufacturer coupon)
- `coupon_match_disc` > 0 (store matches competitor coupon)

### 3.2 Coupon Redemptions
From `coupon_redemptions`, we count how many coupons each household redeemed
and normalize by purchase volume to get "coupons per 100 items purchased"


In [6]:

discount_cols = ['retail_disc', 'coupon_disc', 'coupon_match_disc']

missing_cols = [c for c in discount_cols if c not in transactions.columns]
if missing_cols:
    discount_cols = [c for c in discount_cols if c in transactions.columns]


if discount_cols:
    transactions['has_discount'] = (transactions[discount_cols] > 0).any(axis=1)
else:
    print("⚠️ No discount columns found - assuming no discounts")
    transactions['has_discount'] = False

total_transactions = len(transactions)
discounted_transactions = transactions['has_discount'].sum()
discount_rate = discounted_transactions / total_transactions

summary_df = pd.DataFrame({
    'Metric': ['Total Transactions', 'Transactions with Discount', 'Overall Discount Rate'],
    'Value': [f'{total_transactions:,}', f'{discounted_transactions:,}', f'{discount_rate:.1%}']
})
display(summary_df.set_index('Metric'))

print("\nSample transactions with discounts:")
display(transactions[transactions['has_discount']][
    ['household_id', 'sales_value', 'retail_disc', 'coupon_disc', 'coupon_match_disc']
].head(10))



,Value
Metric,
Total Transactions,"1,469,307"
Transactions with Discount,"746,192"
Overall Discount Rate,50.8%



Sample transactions with discounts:


,household_id,sales_value,retail_disc,coupon_disc,coupon_match_disc
1,900,0.99,0.10,0.00,0.00
2,1228,1.43,0.15,0.00,0.00
3,906,1.50,0.29,0.00,0.00
4,906,2.78,0.80,0.00,0.00
5,906,5.49,0.50,0.00,0.00
6,906,1.50,0.29,0.00,0.00
7,1058,1.88,0.21,0.00,0.00
8,1058,1.50,1.29,0.00,0.00
19,1419,1.00,0.29,0.00,0.00
22,1873,1.50,0.29,0.00,0.00


In [7]:

discount_usage = (
    transactions
    .groupby('household_id')
    .agg(
        total_items=('has_discount', 'size'),
        discounted_items=('has_discount', 'sum'),
        total_sales=('sales_value', 'sum')
    )
    .reset_index()
)

discount_usage['discount_share'] = (
    discount_usage['discounted_items'] / discount_usage['total_items']
)

print("\nDiscount share distribution:")
print(discount_usage['discount_share'].describe())

print("\nTop 10 households by discount usage:")
display(discount_usage.nlargest(10, 'discount_share')[
    ['household_id', 'total_items', 'discounted_items', 'discount_share']
])




Discount share distribution:
count   2469.00
mean       0.50
std        0.11
min        0.00
25%        0.43
50%        0.50
75%        0.57
max        1.00
Name: discount_share, dtype: float64

Top 10 households by discount usage:


,household_id,total_items,discounted_items,discount_share
228,231,2,2,1.00
237,240,1,1,1.00
269,272,1,1,1.00
348,352,1,1,1.00
959,967,1,1,1.00
1166,1176,1,1,1.00
1387,1400,5,5,1.00
2087,2111,362,325,0.90
1657,1674,9,8,0.89
1328,1341,98,87,0.89


In [8]:

coupon_usage = (
    coupon_redemptions
    .groupby('household_id')
    .size()
    .rename('num_coupons_redeemed')
    .reset_index()
)

coupon_summary = pd.DataFrame({
    'Metric': ['Households with Coupons', 'Total Coupons Redeemed', 'Average per Household'],
    'Value': [f'{len(coupon_usage):,}', f'{coupon_usage["num_coupons_redeemed"].sum():,}', f'{coupon_usage["num_coupons_redeemed"].mean():.1f}']
})
display(coupon_summary.set_index('Metric'))

print("\nTop 10 coupon users:")
display(coupon_usage.nlargest(10, 'num_coupons_redeemed'))



,Value
Metric,
Households with Coupons,410
Total Coupons Redeemed,"2,102"
Average per Household,5.1



Top 10 coupon users:


,household_id,num_coupons_redeemed
45,256,30
65,367,30
154,931,29
406,2489,27
290,1823,25
11,67,24
387,2400,23
397,2451,23
101,588,22
252,1591,22


## 4. Define Spending Behavior Metrics

To assess whether promotion-heavy households are high-value or low-value, we calculate:

- **Total annual spending** (`total_sales`)
- **Number of shopping trips** (`num_trips` = distinct baskets)
- **Average basket value** (total_sales / num_trips)
- **Average price per unit** (total_sales / total_quantity) - if quantity data available

These metrics help us distinguish between:
- High-value customers (large baskets, expensive items)
- Low-value customers (small baskets, cheap items)


In [9]:

qty_col = None
for candidate in ['quantity', 'QUANTITY', 'purchase_quantity']:
    if candidate in transactions.columns:
        qty_col = candidate
        break

agg_dict = {
    'sales_value': 'sum',
    'basket_id': pd.Series.nunique,
}

if qty_col:
    agg_dict[qty_col] = 'sum'
else:
    print("⚠️ No quantity column found - cannot calculate avg price per unit")

household_spend = (
    transactions
    .groupby('household_id')
    .agg(agg_dict)
    .rename(columns={
        'sales_value': 'total_sales',
        'basket_id': 'num_trips',
    })
    .reset_index()
)

household_spend['avg_basket_value'] = (
    household_spend['total_sales'] / household_spend['num_trips']
)

if qty_col:
    household_spend.rename(columns={qty_col: 'total_quantity'}, inplace=True)
    household_spend['avg_price_per_unit'] = (
        household_spend['total_sales'] / household_spend['total_quantity'].replace(0, np.nan)
    )
    
    
    zero_qty_count = (household_spend['total_quantity'] == 0).sum()
    if zero_qty_count > 0:
        print(f"  ⚠️ Warning: {zero_qty_count} households have zero total quantity")


print("\nSpending behavior summary:")
print(household_spend[['total_sales', 'num_trips', 'avg_basket_value']].describe())

if 'avg_price_per_unit' in household_spend.columns:
    print("\nAverage price per unit:")
    print(household_spend['avg_price_per_unit'].describe())




Spending behavior summary:
       total_sales  num_trips  avg_basket_value
count      2469.00    2469.00           2469.00
mean       1861.50      63.12             31.86
std        2022.53      69.85             20.72
min           2.00       1.00              2.00
25%         468.85      20.00             17.58
50%        1210.02      43.00             27.30
75%        2536.15      83.00             40.88
max       24879.75     780.00            188.34

Average price per unit:
count   2469.00
mean       1.23
std        1.33
min        0.00
25%        0.03
50%        1.38
75%        2.22
max       25.00
Name: avg_price_per_unit, dtype: float64


In [10]:
print("\nTop 10 households by total spending:")
display(household_spend.nlargest(10, 'total_sales')[
    ['household_id', 'total_sales', 'num_trips', 'avg_basket_value']
])




Top 10 households by total spending:


,household_id,total_sales,num_trips,avg_basket_value
1015,1023,24879.75,375,66.35
1593,1609,16581.36,245,67.68
1440,1453,13376.23,502,26.65
2294,2322,13187.54,178,74.09
1417,1430,12849.31,202,63.61
1101,1111,12368.36,162,76.35
394,400,11801.64,191,61.79
699,707,11491.56,295,38.95
1636,1653,11190.39,338,33.11
1475,1489,10963.69,348,31.50


## 5. Combine Data: Promotions + Spending + Demographics

Now we create a master household-level dataset that includes:
- Promotion usage (discount share + coupon redemptions)
- Spending behavior (total sales, basket value, item prices)
- Demographics (age, income, kids, household size)

This unified dataset enables us to answer our key questions about smart shoppers.


In [11]:

households = (
    household_spend
    .merge(discount_usage[['household_id', 'discount_share', 'discounted_items', 'total_items']], 
           on='household_id', how='left', suffixes=('', '_disc'))
    .merge(coupon_usage, on='household_id', how='left')
    .merge(demographics, on='household_id', how='left')
)

households['num_coupons_redeemed'] = households['num_coupons_redeemed'].fillna(0)
households['discount_share'] = households['discount_share'].fillna(0)


print("\nMerge quality check:")
household_data_df = pd.DataFrame({
    'Data Type': [
        'Households with Spending Data',
        'Households with Demographic Data',
        'Households with Discount Data',
        'Households with Coupon Redemptions'
    ],
    'Count': [
        f"{household_spend['household_id'].nunique():,}",
        f"{households['age'].notna().sum():,}",
        f"{households['discount_share'].notna().sum():,}",
        f"{(households['num_coupons_redeemed'] > 0).sum():,}"
    ]
})
display(household_data_df.set_index('Data Type'))




Merge quality check:


,Count
Data Type,
Households with Spending Data,"2,469"
Households with Demographic Data,801
Households with Discount Data,"2,469"
Households with Coupon Redemptions,410


In [12]:
print("\nSample of unified household data:")
display(households[[
    'household_id', 'total_sales', 'avg_basket_value', 'discount_share', 
    'num_coupons_redeemed', 'income', 'kids_count', 'household_comp'
]].head(15))




Sample of unified household data:


,household_id,total_sales,avg_basket_value,discount_share,num_coupons_redeemed,income,kids_count,household_comp
0,1,2415.56,47.36,0.47,5.00,35-49K,0,2 Adults No Kids
1,2,1024.12,51.21,0.46,0.00,NaN,NaN,NaN
2,3,1026.63,51.33,0.57,0.00,NaN,NaN,NaN
3,4,442.14,24.56,0.48,0.00,NaN,NaN,NaN
4,5,299.67,14.98,0.51,0.00,NaN,NaN,NaN
5,6,3464.87,22.35,0.37,0.00,NaN,NaN,NaN
6,7,1952.37,61.01,0.44,0.00,50-74K,0,2 Adults No Kids
7,8,3080.81,47.40,0.50,1.00,25-34K,1,2 Adults Kids
8,9,620.73,56.43,0.57,0.00,NaN,NaN,NaN
9,10,29.96,29.96,0.50,0.00,NaN,NaN,NaN


In [13]:

households['coupons_per_100_items'] = (
    households['num_coupons_redeemed'] / households['total_items'] * 100
)

print("\nDistribution:")
print(households['coupons_per_100_items'].describe())




Distribution:
count   2469.00
mean       0.08
std        0.25
min        0.00
25%        0.00
50%        0.00
75%        0.00
max        3.24
Name: coupons_per_100_items, dtype: float64
